1. Data Extraction and filtering

In [1]:
# libraries
import pandas as pd
import re
from collections import defaultdict

In [2]:
#function for filtering the whatsapp exported file 
def preprocess(file_path:str)->pd.DataFrame:
    encryption_message="Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n"
    grppermission=r"\s*(.*) changed this group's settings to allow \s*(.*)"
    media_pattern="<Media omitted>"
    email_pattern=r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}"
    url_pattern=r"https?://[^\s]+"
    edited_message="<This message was edited>"
    deleted_message="You deleted this message"
    grp_deleted="This message was deleted"
    null_message="null"
    created_group="created group"
    added_you="added you"
    tagging_pattern=r'@[\w]+'
    noti=r"Your security code with \s*(.*)"
    no_change=r"\s*(.*) changed their phone number to a new number. Tap to message or add the new number."
    vcfmsg=r".*?\.vcf \s*\(file attached\)"
    
    with open(file_path,'r',encoding='utf-8') as f:
        lines=f.readlines()
    filtered_lines=[]
    for line in lines:
        if(encryption_message not in line and
           deleted_message not in line and
           grp_deleted not in line and
           null_message != line.split(" ")[-1] and
           media_pattern not in line and
           created_group not in line and
           added_you not in line and
           not re.search(email_pattern,line) and
           not re.search(url_pattern,line) and 
           not re.search(noti,line) and 
           not re.search(no_change,line) and not re.search(vcfmsg,line) and not re.search(grppermission,line)):
            line=line.replace(edited_message,"").strip()
            line=re.sub(tagging_pattern,"",line).strip()
            filtered_lines.append(line)
    content='\n'.join(filtered_lines)
    content=content.replace('\u202f'," ")#clean invisible lines
    # print(content)
    
    #regex for message 
    
    pattern= r'(\d{2}/\d{2}/\d{4}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - ([^:]+):\s*(.*)'
    pattern24= r'(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}) - ([^:]+):\s*(.*)'
    messages=re.findall(pattern,content)
    messages24=re.findall(pattern24,content)
    # print(messages)
    # print(messages24)
    combined=[(f"{d}, {t}",s,m) for d,t,s,m in messages]
    combined+=[(f"{d}, {t}",s,m) for d,t,s,m in messages24]
    df=pd.DataFrame(combined,columns=['timestamp','sender','message'])
    # df.insert(0,'id',range(0,len(df)))
    # df.set_index('id', inplace=True)
    #handling timestamp
    
    timestamps=[]
    for timestamp in df['timestamp']:
        try:
            timestamp=pd.to_datetime(
                timestamp,errors='coerce'
            )
        except Exception as e:
            print("error",e)
            timestamp=pd.NaT
        timestamps.append(timestamp)
    df['timestamp']=timestamps
    return df

In [3]:
file=preprocess("./dataset/WhatsApp Chat with Didi Jio.txt")
file2=preprocess("./dataset/WhatsApp Chat with Kartikey Dubey.txt")
file3=preprocess("./dataset/WhatsApp Chat with Akansha Shishodia.txt")
file4=preprocess("./dataset/WhatsApp Chat with I.T. 3rd year unofficial.txt")
file5=preprocess("./dataset/WhatsApp Chat with Hack it.txt")
file6=preprocess("./dataset/WhatsApp Chat with IT -III YEAR (LATERAL ENTRY) 2023-2026.txt")
file7=preprocess("./dataset/WhatsApp Chat with Mami.txt")
file8=preprocess("./dataset/WhatsApp Chat with IT Information.txt")
file9=preprocess("./dataset/WhatsApp Chat with Placement Information batch 2023.txt")
file10=preprocess("./dataset/whatsapp-chat-data.txt")
dataset=pd.concat([file,file2,file3,file4,file5,file6,file7,file8,file9,file10])
dataset.to_csv("./dataset/chat.csv",index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_4992\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(
C:\Users\Admin\AppData\Local\Temp\ipykernel_4992\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M am format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(
C:\Users\Admin\AppData\Local\Temp\ipykernel_4992\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M pm format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(


In [4]:
dataset.shape

(38980, 3)

output

In [5]:
text_seq=(" ").join(dataset['message'])
with open('./output/text_seq.txt','w',encoding='utf-8') as f:
    f.write(text_seq)

manual bpe

In [6]:
# def preprocess_text(text_seq):
#     text_seq.lower()
#     texta_seq=re.sub(r'[^\w\s]','',text_seq)
#     texts=text_seq.split()
#     return texts
# #intial vocab
# def get_vocab(corpus):
#     vocab = defaultdict(int) # defautdict is a data structure which help in avoiding key error if unique word enter then the intialize its freq to zero so we can easily increase it
#     for word in corpus:
#         chars = ' '.join(list(word)) + ' </w>'  # Add </w> to mark end of word
#         vocab[chars] += 1
#     return vocab
# # vocab=get_vocab(texts)
# # print(len(vocab))
# #count freq of adjecent symbol pairs
# def get_stats(vocab):
#     pairs=defaultdict(int)
#     for word,freq in vocab.items():
#         symbols=word.split()
#         for i in range(len(symbols)-1):
#             pair=(symbols[i],symbols[i+1])
#             pairs[pair]+=freq
#     return max(pairs,key=pairs.get), max(pairs.values())
# # pair=get_stats(vocab)
# # print(pair)
# #merge
# def merge_vocab(pair, vocab):
#     pattern = re.escape(' '.join(pair))
#     replacement = ''.join(pair)
#     new_vocab = {}
#     for word in vocab:
#         new_word = re.sub(pattern, replacement, word)
#         new_vocab[new_word] = vocab[word]
#     return new_vocab
# # result=merge_vocab(pair[0],vocab)
# # print(result)
# def bpe(corpus, num_merges=10):
#     vocab = get_vocab(corpus)
#     print("Initial Vocabulary:")
#     for word in vocab:
#         print(word)
#     print("\nStarting BPE Merges:\n")
#     for i in range(num_merges):
#         pair, freq = get_stats(vocab)
#         if not pair:
#             break
#         print(f"Merge #{i+1}: {pair} -> {''.join(pair)} (Frequency: {freq})")
#         vocab = merge_vocab(pair, vocab)
#     return vocab
# corpus=preprocess_text(text_seq)
# result=bpe(corpus,num_merges=11)
# result

In [7]:
# Automate BPE
import sys
sys.path.append('./')
from minbpe import BasicTokenizer

vocab_size=1234
tokenizer = BasicTokenizer()
tokenizer.train(text_seq,vocab_size)
vocab=tokenizer.vocab
vocab
tokenizer.encode('Shree ganesh')
# isko todega phir token wala no assign karega
tokenizer.decode([83, 104, 524, 103, 271, 364, 104])
max_vocab_id = list(tokenizer.vocab.keys())[-1]
print(max_vocab_id)
tokenizer.special_tokens = {
    "<|startoftext|>": max_vocab_id + 1,
    "<|separator|>": max_vocab_id + 2,
    "<|endoftext|>": max_vocab_id + 3,
    "<|unk|>": max_vocab_id + 4,
    "<|padding|>": max_vocab_id + 5,
}

# for fine tuning special tokens
# output
print(len(tokenizer.encode(text_seq)))
tokenizer.save(file_prefix="./output/my_tokenizer")

1233
392160


In [8]:
def get_vocab_size(tokenizer:BasicTokenizer):
    vocab_size=len(tokenizer.vocab)
    special_token=len(tokenizer.special_tokens)
    return vocab_size+special_token

In [9]:
get_vocab_size(tokenizer)

1239

Transformer

In [10]:
import torch
torch.manual_seed(4893)
# hyperparameters
# Transformer model hyperparameters
block_size = 256     # Maximum context length for each input sequence
n_embd = 256        # Embedding size (dimension of each token's vector)
n_head = 4         # Number of attention heads
n_layer = 4          # Number of transformer layers
dropout = 0.2        # Dropout rate for regularization
vocab_size=get_vocab_size(tokenizer)
device='cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
#head 
from typing import Optional, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as f

class Head(nn.Module):#small brain
    """ One Head of Self-Attention"""
    def __init__(self,head_size:int):
        super().__init__()# It connects code to PyTorch’s engine.
        # A Query vector: what am I looking for?
        # A Key vector: what do I offer?
        # A Value vector: what info do I carry?
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))#“Only look at this word and the ones before it — not ahead.”
        self.dropout=nn.Dropout(dropout)#randomly turning off a few brain cells while learning.
        #To avoid overthinking or memorizing everything. It's a trick to help the model generalize better.
        self.head_size=head_size
    def forward(self,x:torch.Tensor)->torch.Tensor:
        _,T,_=x.shape#[batch_size, T, n_embd]
        k=self.key(x)#Transforms the input into a key vector for each word.
        q=self.query(x)#Transforms the input into a query vector for each word.
        weights=q@k.transpose(-2,-1)*k.shape[-1]**-0.5
        #“How well does this word (query) match with every other word (key)?”
        weights=weights.masked_fill(self.tril[:T,:T]==0,float('-inf'))#This hides the future.
        weights=f.softmax(weights,dim=-1)#"How much should I pay attention to each word?"
        weights=self.dropout(weights)#Drop some attention values to avoid overfitting (same as before).
        v=self.value(x)#Now we get the actual information from each word (value vector).
        out=weights@v#“Give me a mix of values, based on how much I should pay attention to each one.”
        return out

In [12]:
# multihead attention
# The idea is to have multiple heads of attention, each focusing on different parts of the input.
class MultiHeadAttention(nn.Module):
    """Multiple heads of self attention in parallel"""
    def __init__(self,num_heads:int,head_size:int):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in  range(num_heads)]) #This creates a list of heads, each with its own set of parameters. 
        self.projection=nn.Linear(num_heads*head_size,n_embd) #This combines the outputs of all heads into a single vector.
        self.dropout=nn.Dropout(dropout) #This randomly turns off some neurons to prevent overfitting.
    def forward(self,x:torch.Tensor)->torch.Tensor:
        out=torch.cat([h(x) for h in self.heads],dim=-1)    #This concatenates the outputs of all heads into a single tensor.
        out=self.projection(out) #This combines the outputs of all heads into a single vector.
        out=self.dropout(out) #This randomly turns off some neurons to prevent overfitting.
        return out

In [13]:
# Block
# A block is a self-attention layer followed by a feed-forward layer
class FeedForward(nn.Module):
    """A simple linear layer followed by a non-linearity"""
    def __init__(self,n_embd:int):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd,n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4,n_embd),
            nn.Dropout(dropout)
        )
    def forward(self,x:torch.Tensor)->torch.Tensor:
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer Block: Self-Attention + Feed Forward"""
    def __init__(self,n_embd:int,n_head:int):
        super().__init__()
        head_size=n_embd//n_head
        self.self_attention=MultiHeadAttention(n_head,head_size)
        self.feed_forward=FeedForward(n_embd)
        self.layer_norm_1=nn.LayerNorm(n_embd)
        self.layer_norm_2=nn.LayerNorm(n_embd)
    def forward(self,x:torch.Tensor)->torch.Tensor:
        x=self.layer_norm_1(x+self.self_attention(x))
        x=self.layer_norm_2(x+self.feed_forward(x))
        return x

In [14]:
# Assembling the model
class GPTLanguageModel(nn.Module):
    """ The full GPT Language Model"""
    def __init__(self):
        # vocab_size: size of the vocabulary
        # n_embd: size of the embedding vector
        # n_layer: number of transformer blocks
        # n_head: number of attention heads
        # block_size: maximum context length for each input sequence
        # This is the main class that combines all the components of the model. It initializes the token and position embeddings, the transformer blocks, and the final linear layer.
        super().__init__()
        self.token_embedding=nn.Embedding(vocab_size,n_embd)
        self.position_embedding=nn.Embedding(block_size,n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd,n_head) for _ in range(n_layer)])
        self.final_layer_norm=nn.LayerNorm(n_embd)
        self.final_linear_layer=nn.Linear(n_embd,vocab_size)
        self.apply(self._init_weights)
        
    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            # torch.nn.init.zeros_(module.weight)
    def forward(self,input_tokens:torch.Tensor,targets:Optional[torch.tensor]=None)->torch.Tensor:
        """ forward pass of the model
        input_tokens: Tensor of input token indices of shape (batch_size, sequence_length)
        targets: target token ids (for training)
        returns: tupules of (logits, loss) where logits of shape (batch_size, block_size, vocab_size) and loss is the optional cross-entropy loss
        """
        B,T=input_tokens.shape
        token_embeddings=self.token_embedding(input_tokens)
        position_embeddings=self.position_embedding(torch.arange(T,device=device)) #This creates a tensor of position indices for the input sequence.
        x=token_embeddings+position_embeddings #This adds the token and position embeddings together. shape (B,T,n_embd)
        x=self.blocks(x) #This applies the transformer blocks to the input embeddings. shape (B,T,n_embd)
        x=self.final_layer_norm(x) #This applies layer normalization to the output of the transformer blocks. shape (B,T,n_embd)
        logits=self.final_linear_layer(x) #This applies a linear layer to the output of the transformer blocks. shape (B,T,vocab_size) 
        # logits=logits.view(B*T,-1) #This reshapes the output to (B*T,vocab_size) for loss calculation.
        if targets is None:
            loss=None
        else:
            B,T,V=logits.shape
            logits=logits.view(B*T,V)
            targets=targets.view(B*T)
            loss=f.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,input_tokens:torch.Tensor,max_new_tokens:int)->torch.Tensor:
        # This function generates new tokens based on the input tokens. 
        # It uses the model to predict the next token in the sequence and appends it to the input tokens.
        # This process is repeated for the specified number of new tokens.
        # input_tokens: Tensor of input token indices of shape (batch_size, sequence_length)
        # max_new_tokens: number of new tokens to generate
        # returns: Tensor of generated token indices of shape (batch_size, sequence_length + max_new_tokens)
        
        for _ in range(max_new_tokens): 
            cropped_input=input_tokens[:,-block_size:] #This ensures that the input sequence does not exceed the maximum context length.
            # This is the main part of the generation process. It uses the model to predict the next token in the sequence.
            logits,_=self(cropped_input)
            # logits=logits[:,-1,:] #This selects the last token in the sequence for prediction.
            logits=logits[:,-1,:] #This selects the last token in the sequence for prediction.
            probs=f.softmax(logits,dim=-1) #This applies the softmax function to the logits to get probabilities for each token in the vocabulary.
            idx_next=torch.multinomial(probs,num_samples=1) #This samples the next token from the predicted probabilities.
            input_tokens=torch.cat((input_tokens,idx_next),dim=1) #This appends the predicted token to the input sequence.
        return input_tokens 

In [15]:
model=GPTLanguageModel()
model.to(device)
print(sum(p.numel() for p in model.parameters())/1e6)

3.857623


In [16]:
batch_size=64
seq_len=block_size=256
x=torch.randint(0,vocab_size,(batch_size,seq_len),device=device)
x=x.to(device)
logits,loss=model(x)
print(logits.shape,loss)

torch.Size([64, 256, 1239]) None


In [19]:
def print_summary(model:torch.nn.Module, indent:str=''):
    """Prints a summary of the model architecture and the number of parameters in each layer."""
    for name, child in model.named_children():
        params= sum(p.numel() for p in child.parameters())
        print(f"{indent}|--{name}: {child.__class__.__name__}({params:,}parameters)")
        print_summary(child, indent + '    ')
print_summary(model)
def get_model_stats(model:torch.nn.Module):
    stats=[]
    for name,module in model.named_modules():
        if len(list(module.children()))==0:
            params=sum(p.numel() for p in module.parameters())
            stats.append({'Layer Name':name,'Type':module.__class__.__name__,'Parameters':params,'Trainable':sum(p.numel() for p in module.parameters() if p.requires_grad)})
    return pd.DataFrame(stats)
        
stats_df=get_model_stats(model)
stats_df

|--token_embedding: Embedding(317,184parameters)
|--position_embedding: Embedding(65,536parameters)
|--blocks: Sequential(3,155,968parameters)
    |--0: Block(788,992parameters)
        |--self_attention: MultiHeadAttention(262,400parameters)
            |--heads: ModuleList(196,608parameters)
                |--0: Head(49,152parameters)
                    |--key: Linear(16,384parameters)
                    |--query: Linear(16,384parameters)
                    |--value: Linear(16,384parameters)
                    |--dropout: Dropout(0parameters)
                |--1: Head(49,152parameters)
                    |--key: Linear(16,384parameters)
                    |--query: Linear(16,384parameters)
                    |--value: Linear(16,384parameters)
                    |--dropout: Dropout(0parameters)
                |--2: Head(49,152parameters)
                    |--key: Linear(16,384parameters)
                    |--query: Linear(16,384parameters)
                    |--value: 

,Layer Name,Type,Parameters,Trainable
0,token_embedding,Embedding,317184,317184
1,position_embedding,Embedding,65536,65536
2,blocks.0.self_attention.heads.0.key,Linear,16384,16384
3,blocks.0.self_attention.heads.0.query,Linear,16384,16384
4,blocks.0.self_attention.heads.0.value,Linear,16384,16384
...,...,...,...,...
95,blocks.3.feed_forward.net.3,Dropout,0,0
96,blocks.3.layer_norm_1,LayerNorm,512,512
97,blocks.3.layer_norm_2,LayerNorm,512,512
98,final_layer_norm,LayerNorm,512,512
